# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import string
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
num_of_cities = len(cities)
print(num_of_cities)

638


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it's being processed (with the city number and city name).

In [3]:
# Openweathermap api url
url = "http://api.openweathermap.org/data/2.5/weather"
units = 'imperial'
params = {'units': units,
          'appid': weather_api_key}

print("""Beginning Data Retrieval
-----------------------------""")
d = []
for idx, city in enumerate(cities, 1):
    # 60 calls/min = 1 call per second
    time.sleep(1)
    
    # set city param
    params['q'] = city
    print(f"Processing Record {idx} of {num_of_cities} | {string.capwords(city)}")
    response = requests.get(url=url, params=params)
    data = response.json()
    # check if city not found
    if data['cod'] == '404':
        print(f"{string.capwords(city)} not found. Skipping...")
        continue
    
    # create new dict entry
    d.append(
        {
            'City': data['name'],
            'Lat': data['coord']['lat'],
            'Lng': data['coord']['lon'],
            'Max_Temp': data['main']['temp_max'],
            'Humidity': data['main']['humidity'],
            'Cloudiness': data['clouds']['all'],
            'Wind_Speed': data['wind']['speed'],
            'Country': data['sys']['country'],
            'Date': pd.to_datetime(data['dt'], unit='s')
        }
    )

print("""-----------------------------
Data Retrieval Complete
-----------------------------""")

Beginning Data Retrieval
-----------------------------
Processing Record 1 of 638 | Taolanaro
Taolanaro not found. Skipping...
Processing Record 2 of 638 | Xuddur
Processing Record 3 of 638 | Mahebourg
Processing Record 4 of 638 | Bacuit
Bacuit not found. Skipping...
Processing Record 5 of 638 | Urumqi
Processing Record 6 of 638 | Grajau
Processing Record 7 of 638 | Saskylakh
Processing Record 8 of 638 | Mataura
Processing Record 9 of 638 | Port Lincoln
Processing Record 10 of 638 | Kodiak
Processing Record 11 of 638 | Nizhneyansk
Nizhneyansk not found. Skipping...
Processing Record 12 of 638 | Akonolinga
Processing Record 13 of 638 | Cape Town
Processing Record 14 of 638 | Norman Wells
Processing Record 15 of 638 | Lufilufi
Processing Record 16 of 638 | Acarau
Processing Record 17 of 638 | Saint-marc
Processing Record 18 of 638 | Juifang
Juifang not found. Skipping...
Processing Record 19 of 638 | Port Blair
Processing Record 20 of 638 | Kapaa
Processing Record 21 of 638 | Lac Du Bonn

Processing Record 192 of 638 | Manzanillo
Processing Record 193 of 638 | Bambanglipuro
Processing Record 194 of 638 | Wakefield
Processing Record 195 of 638 | Klaksvik
Processing Record 196 of 638 | Dunedin
Processing Record 197 of 638 | Kirakira
Processing Record 198 of 638 | Severo-kurilsk
Processing Record 199 of 638 | Vaitupu
Vaitupu not found. Skipping...
Processing Record 200 of 638 | Tsihombe
Tsihombe not found. Skipping...
Processing Record 201 of 638 | Agdas
Processing Record 202 of 638 | Kieta
Processing Record 203 of 638 | Losinyy
Processing Record 204 of 638 | Magomeni
Processing Record 205 of 638 | Marawi
Processing Record 206 of 638 | Lokosovo
Processing Record 207 of 638 | Angra
Angra not found. Skipping...
Processing Record 208 of 638 | Sao Pedro Do Sul
Processing Record 209 of 638 | Richards Bay
Processing Record 210 of 638 | Billings
Processing Record 211 of 638 | Sitka
Processing Record 212 of 638 | Acapulco
Processing Record 213 of 638 | Ketchikan
Processing Record 

Processing Record 383 of 638 | Marrakesh
Processing Record 384 of 638 | Kaitangata
Processing Record 385 of 638 | Vondrozo
Processing Record 386 of 638 | Treinta Y Tres
Processing Record 387 of 638 | Turukhansk
Processing Record 388 of 638 | Axim
Processing Record 389 of 638 | Dubbo
Processing Record 390 of 638 | Hvide Sande
Processing Record 391 of 638 | Contamana
Processing Record 392 of 638 | Kargil
Processing Record 393 of 638 | Sarakhs
Processing Record 394 of 638 | Rancho Palos Verdes
Processing Record 395 of 638 | Villa Guerrero
Processing Record 396 of 638 | Esmeraldas
Processing Record 397 of 638 | Provideniya
Processing Record 398 of 638 | Palabuhanratu
Palabuhanratu not found. Skipping...
Processing Record 399 of 638 | Khatanga
Processing Record 400 of 638 | Mahadday Weyne
Mahadday Weyne not found. Skipping...
Processing Record 401 of 638 | Takoradi
Processing Record 402 of 638 | Jerantut
Processing Record 403 of 638 | Chokwe
Processing Record 404 of 638 | Kisangani
Processi

Processing Record 571 of 638 | Tartagal
Processing Record 572 of 638 | Kargopol
Processing Record 573 of 638 | Bati
Processing Record 574 of 638 | Chulman
Processing Record 575 of 638 | Goure
Processing Record 576 of 638 | Iguala
Processing Record 577 of 638 | Tessalit
Processing Record 578 of 638 | Flinders
Processing Record 579 of 638 | Colon
Processing Record 580 of 638 | Sao Felix Do Xingu
Processing Record 581 of 638 | Mastic Beach
Processing Record 582 of 638 | Sisimiut
Processing Record 583 of 638 | Seoul
Processing Record 584 of 638 | Chifeng
Processing Record 585 of 638 | Khani
Processing Record 586 of 638 | Clarence Town
Processing Record 587 of 638 | Nguruka
Processing Record 588 of 638 | Mount Isa
Processing Record 589 of 638 | Husavik
Processing Record 590 of 638 | Ust-kut
Processing Record 591 of 638 | Farafangana
Processing Record 592 of 638 | Nogliki
Processing Record 593 of 638 | Mimarsinan
Processing Record 594 of 638 | Iralaya
Processing Record 595 of 638 | Asau
Proc

### If API Calls Was Run, Run This Cell

In [4]:
weather_df = pd.DataFrame(d)
weather_df.to_csv('weather.csv', index=False)
weather_df.head()

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,Oddur,4.12,43.89,82.53,44,98,22.21,SO,2020-07-23 13:43:00
1,Mahébourg,-20.41,57.70,69.80,64,75,19.46,MU,2020-07-23 13:46:50
2,Ürümqi,43.80,87.60,80.60,32,75,4.47,CN,2020-07-23 13:46:53
3,Grajaú,-5.82,-46.14,88.41,39,100,8.90,BR,2020-07-23 13:46:54
4,Saskylakh,71.92,114.08,53.13,92,99,12.93,RU,2020-07-23 13:42:06


### If Accessing Saved Data, Run This Cell

In [5]:
weather_df = pd.read_csv('weather.csv')
weather_df.head()

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,Oddur,4.12,43.89,82.53,44,98,22.21,SO,2020-07-23 13:43:00
1,Mahébourg,-20.41,57.70,69.80,64,75,19.46,MU,2020-07-23 13:46:50
2,Ürümqi,43.80,87.60,80.60,32,75,4.47,CN,2020-07-23 13:46:53
3,Grajaú,-5.82,-46.14,88.41,39,100,8.90,BR,2020-07-23 13:46:54
4,Saskylakh,71.92,114.08,53.13,92,99,12.93,RU,2020-07-23 13:42:06


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [6]:
# Get initial data length
data_len = len(weather_df)
weather_df = weather_df.drop(weather_df[weather_df.Humidity > 100].index)
print(f"Removed {data_len - len(weather_df)} cities with Humidity values greater than 100%")

Removed 0 cities with Humidity values greater than 100%


In [7]:
# Write cleaned csv to file
weather_df.to_csv('weather.csv', index=False)

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots

In [ ]:
# Create Northern and Southern Hemisphere DataFrames

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression